## Load Data and Preprocess

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations, chain


df = pd.read_csv("german_data_credit.csv")

## Replace string

In [ ]:
columns_to_map = ["checking-account", "savings-account", "employment-since"]
for col in columns_to_map:
    df[col] = df[col].str.replace('<= <', " to ")
    df[col] = df[col].str.replace('DM', "debit memo")

## Random Sample 70% Training

In [ ]:
train_df = df.sample(frac=0.7, random_state=1)
test_df = df.drop(index=train_df.index)
print(len(test_df))
print(len(train_df))

In [ ]:
test_df["sex"].value_counts()

## Define Prompt

In [ ]:
prompt = "Your task is to determine if the credit risk of a people is good or bad according to the input attributes. Return your answer: 1(Good credit) or 0(Bad credit)\n\
Here are four examples in the next triple quotes:\n\
\"\"\"1. *<EXAMPLE_0>*\n\
2. *<EXAMPLE_1>*\n\
3. *<EXAMPLE_2>*\n\
4. *<EXAMPLE_3>*\"\"\"\n\
A description of the input attributes is in the following quotes.\n\
\"checking-account: The status of existing checking account\n\
duration: The duration of the credit (month)\n\
credit-history: The credit history\n\
purpose: Purpose (car, furniture, education, etc.)\n\
credit-amount: Credit amount\n\
savings-account: Savings account/bonds\n\
employment-since: Present employment since\n\
installment-rate: The installment rate in percentage of disposable income\n\
other-debtors: Other debtors/guarantors\n\
residence-since: Present residence since\n\
property: Property\n\
age: The age of the individual\n\
other-installment: Other installment plans\n\
housing: Housing (rent, own, for free)\n\
existing-credits: Number of existing credits at this bank\n\
job: Job (unemployed, (un)skilled, management)\n\
number-people-provide-maintenance-for: Number of people being liable to provide maintenance for\n\
telephone: Telephone number\n\
foreign-worker: Is the individual a foreign worker?\n\
sex: Sex (male, female)\n\
marital-status: Marital status\"\n\
<Inputs>: *?*\n\
<Answer>: "

# Prepare OpenAI API

In [ ]:
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

# load key
openai.api_key = open("<your_openAI_key_path>", "r").read().strip('\n')

In [ ]:
from typing import List
from tqdm import tqdm


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def single_request(request: str) -> str:
    # call openAI chat completion API
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": request}],
        temperature=0
    )
    reply_msg = completion.choices[0].message.content
    return reply_msg


def batch_requests(requests: List[str]) -> List[str]:
    reply_list = []
    for request in tqdm(requests):
        reply_list.append(single_request(request))
    return reply_list

# Task 0: no sensitive feature

### Sample examples for training examples

In [ ]:
task_id = 1

task_example_0_M = train_df[(train_df["sex"] == "male") & (train_df["class-label"] == 0)].sample(n=1, random_state=1)
task_example_1_M = train_df[(train_df["sex"] == "male") & (train_df["class-label"] == 1)].sample(n=1, random_state=1)
task_example_0_F = train_df[(train_df["sex"] == "female") & (train_df["class-label"] == 0)].sample(n=1, random_state=1)
task_example_1_F = train_df[(train_df["sex"] == "female") & (train_df["class-label"] == 1)].sample(n=1, random_state=1)

task_example_list = [task_example_0_M, task_example_1_M, task_example_0_F, task_example_1_F]

### Prepare examples

In [ ]:
task_prompt = prompt
question = ""

counter = 0
for example in task_example_list:
    for index, row in example.iterrows():
        sample = "<Inputs>: "
        question_str = question
        answer_str = "<Answer>: "
        for i, col in enumerate(example.columns):
            if col != "class-label":
                sample += f"{col}: {row[col]}, "
            else:
                answer_str += f"{row[col]}"
        sample = sample.strip()[:-1] + "\n" + question_str + answer_str
        task_prompt = task_prompt.replace(f"*<EXAMPLE_{counter}>*", sample)
        counter += 1
print(task_prompt)     

### Prepare request strings

In [11]:
counter = 0

task_requests = []

for index, row in test_df.iterrows():
    sample = ""
    for i, col in enumerate(df.columns):
        if col != "class-label":
            sample += f"{col}: {row[col]}, "
    
    request = task_prompt.replace("*?*", sample)
    task_requests.append(request)
print(task_requests[0])

Your task is to determine if the credit risk of a people is good or bad according to the input attributes. Return your answer: 1(Good credit) or 0(Bad credit)
Here are four examples in the next triple quotes:
"""1. <Inputs>: checking-account: 0  to 200 debit memo, duration: 24, credit-history: no credits taken, purpose: business, credit-amount: 4241, savings-account: <100 debit memo, employment-since: 1 to  4 years, installment-rate: 1, other-debtors: none, residence-since: 4, property: real estate, age: 36, other-installment: none, housing: own, existing-credits: 3, job: unskilled - resident, numner-people-provide-maintenance-for: 1, telephone: yes, foreign-worker: yes, sex: male, marital-status: divorced/separated
<Answer>: 0
2. <Inputs>: checking-account: no account, duration: 12, credit-history: critical account, purpose: radio/television, credit-amount: 976, savings-account: no savings account, employment-since: >=7 years, installment-rate: 4, other-debtors: none, residence-since:

### Call API

In [ ]:
import time

start_time = time.time()
task_response = batch_requests(task_requests)

print(f"--- {len(task_requests)} requests in {time.time() - start_time} seconds ---")

In [ ]:
# task_df = test_df.copy()
task_df = pd.read_csv("German_response_task_0_to_5.csv")
task_df[f"task_{task_id}_response"] = task_response
task_df[f"task_{task_id}_response"]= task_df[f"task_{task_id}_response"].astype(int)
task_df.to_csv("German_response_task_0_to_5.csv", index=False, sep=",")

In [ ]:
task_df.head()

## Filter out rows with response only

In [ ]:
with_rsp = task_df[task_df[f"task_{task_id}_response"].isin([0, 1])].copy()
with_rsp

In [ ]:
print(f"Response Rate: {len(with_rsp) / len(task_df)}")

## Evaluation Metrics Function

In [ ]:
from collections import defaultdict
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


def statistical_parity(data: pd.DataFrame, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = {}
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        y_hat_1 = data_group_a[data_group_a[y_hat_col] == 1]
        result_dict[sens_val] = len(y_hat_1) / len(data_group_a)
    return result_dict


def equal_opportunity(data: pd.DataFrame, y_col, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = {}
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        y_1 = data_group_a[data_group_a[y_col] == 1].copy()
        y_and_y_hat_1 = y_1[y_1[y_hat_col] == 1].copy()
        result_dict[sens_val] = len(y_and_y_hat_1) / len(y_1)
    return result_dict


def equalize_odds(data: pd.DataFrame, y_col, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = defaultdict(dict)
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        y_1 = data_group_a[data_group_a[y_col] == 1].copy()
        y_0 = data_group_a[data_group_a[y_col] == 0].copy()
        y_and_y_hat_1 = y_1[y_1[y_hat_col] == 1].copy()
        y_hat_1_y_0 = y_0[y_0[y_hat_col] == 1].copy()

        result_dict[sens_val]["tpr"] = len(y_and_y_hat_1) / len(y_1)
        result_dict[sens_val]["fpr"] = len(y_hat_1_y_0) / len(y_0)
    return result_dict


def accuracy_report(data: pd.DataFrame, y_col, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = defaultdict(dict)
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        correct = data_group_a[((data_group_a[y_col] == 1) & (data_group_a[y_hat_col] == 1)) | ((data_group_a[y_col] == 0) & (data_group_a[y_hat_col] == 0))]
        result_dict[sens_val] = len(correct) / len(data_group_a)
        
    all_correct = data[((data[y_col] == 1) & (data[y_hat_col] == 1)) | ((data[y_col] == 0) & (data[y_hat_col] == 0))]
    result_dict["overall"] = len(all_correct) / len(data)
    return result_dict


def auc(data: pd.DataFrame, y_col, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = defaultdict(dict)
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        y = data_group_a[y_col].tolist()
        y_hat = data_group_a[y_hat_col].tolist()
        result_dict[sens_val] = roc_auc_score(y, y_hat)
        
    all_y = data[y_col].tolist()
    all_y_hat = data[y_hat_col].tolist()
    result_dict["overall"] = roc_auc_score(all_y, all_y_hat)
    return result_dict


def f1(data: pd.DataFrame, y_col, y_hat_col, sens_col):
    sens_vals = data[sens_col].unique().tolist()
    result_dict = defaultdict(dict)
    for sens_val in sens_vals:
        data_group_a = data[data[sens_col] == sens_val].copy()
        y = data_group_a[y_col].tolist()
        y_hat = data_group_a[y_hat_col].tolist()
        result_dict[sens_val] = f1_score(y, y_hat)
        
    all_y = data[y_col].tolist()
    all_y_hat = data[y_hat_col].tolist()
    result_dict["overall"] = f1_score(all_y, all_y_hat)
    return result_dict

In [ ]:
stat_parity = statistical_parity(with_rsp, f"task_{task_id}_response", "sex")
print(f"statistical parity {stat_parity}")

In [ ]:
equal_op = equal_opportunity(with_rsp, "class-label", f"task_{task_id}_response", "sex")
print(f"equal opportunity {equal_op}")

In [ ]:
equal_odds = equalize_odds(with_rsp, "class-label", f"task_{task_id}_response", "sex")
print(f"equal opportunity {equal_odds}")

In [ ]:
accuracy = accuracy_report(with_rsp, "class-label", f"task_{task_id}_response", "sex")
print(f"accuracy report {accuracy}")

In [ ]:
f1_result = f1(with_rsp, "class-label", f"task_{task_id}_response", "sex")
print(f"f1 {f1_result}")

In [ ]:
auc_result = auc(with_rsp, "class-label", f"task_{task_id}_response", "sex")
print(f"auc {auc_result}")